# Code used to save simulations as .csv files in order to reduce developement time wasted on running simulations over and over again. Csv files were not uploaded since they weigh > 1 gigabyte.

In [20]:
import wntr
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import os

In [21]:
data_file = 'Walkerton_v1.inp'

# Save 2-week simulation without leaks

In [22]:
wn = wntr.network.WaterNetworkModel(data_file)
sim = wntr.sim.WNTRSimulator(wn)
sim_results = sim.run_sim()

c:\users\pc\anaconda3\envs\wodociagi\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [23]:
final = pd.concat([sim_results.node['pressure'], sim_results.link['flowrate']], axis=1)

In [24]:
final.to_csv(os.path.join(os.getcwd(),'simulations', 'no_leaks', 'sim_nL.csv'), index=False)

In [25]:
pd.read_csv(os.path.join(os.getcwd(),'simulations', 'no_leaks', 'sim_nL.csv'))

,J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,...,P548,P549,P551,P553,PMP51,PMP63,PMP72,FCV51,FCV63,FCV72
0,36.011416,30.044852,29.924852,31.958688,32.148688,45.531102,31.841401,28.841400,33.861399,43.310270,...,0.005835,-0.001642,0.001183,-0.000037,0.020833,0.0,0.0434,0.020833,0.0,0.0434
1,36.643471,30.803467,30.683467,32.673468,32.863468,45.903506,32.473456,29.473455,34.493454,43.604068,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
2,36.438482,30.598478,30.478478,32.468479,32.658479,45.698517,32.268468,29.268467,34.288465,43.399079,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
3,36.233493,30.393490,30.273490,32.263490,32.453490,45.493529,32.063479,29.063478,34.083477,43.194090,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
4,36.028505,30.188501,30.068501,32.058502,32.248502,45.288540,31.858490,28.858489,33.878488,42.989102,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,35.237505,29.400005,29.280005,31.269126,31.459126,44.493960,31.067235,28.067200,33.087205,42.193966,...,0.002556,-0.000554,-0.000079,-0.000022,0.000000,0.0,0.0000,0.000000,0.0,0.0000
333,35.028346,29.195079,29.075079,31.062724,31.252724,44.277401,30.858022,27.857987,32.877978,41.976184,...,0.001838,-0.000396,-0.000148,-0.000022,0.000000,0.0,0.0000,0.000000,0.0,0.0000
334,34.714314,28.881794,28.761794,30.749179,30.939179,43.961869,30.544018,27.543951,32.564005,41.660273,...,0.001705,-0.000365,-0.000160,-0.000022,0.000000,0.0,0.0000,0.000000,0.0,0.0000
335,34.415027,28.572417,28.452417,30.443311,30.633311,43.680726,30.245013,27.245012,32.265011,41.383795,...,0.002541,-0.000559,0.000204,-0.000009,0.000000,0.0,0.0000,0.000000,0.0,0.0000


# Save simulations with leaks

In [52]:
from dataclasses import dataclass
unknown_pressure = -1
junctions_with_no_edges = ['JPMP51', 'JPMP63', 'JPMP72', 'R5', 'R6', 'R7']	

@dataclass
class Junction:
	name: str
	x: float
	y: float
	pressure: float

@dataclass
class Pipe:
	name: str
	node1: str
	node2: str
	length: float
	status: str

def get_node_data(data_file: str) -> [list, list]:
	"""
	Parses .inp file and finds data on pipes and junctions, creates Pipe and Junction objects
	Arguments: 	.inp file (str)
	Output: 	[junctions, pipes] where pipes is a list of pipes and junctions is a list of junctions
	"""
	junctions_found = False
	pipes_found = False
	junctions = []
	pipes = []

	with open(data_file, 'r') as file:
		content = iter(file.readlines())														# get all content into single var
		for line in content:
			current_line = line.split()															# get rid of whitespaces

			if current_line == ['[PIPES]']:														# look for pipes info section
					pipes_found = True
					next(content)																# skip a line (column headers)
					continue
			if current_line == ['[COORDINATES]']:												# look for pipes info section
					junctions_found = True
					next(content)																# skip a line (column headers)
					continue

			if pipes_found and not junctions_found:												# if in pipes section
				if current_line == []:															# if end of pipes section
					pipes_found = False															# pipes section ends with \n
					continue
				else:
					name, node1, node2, length, _, _, _, status, _ = current_line
					pipes.append(Pipe(name=name,
						node1=node1,
						node2=node2,
						length=length,
						status=status))

			if junctions_found and not pipes_found:												# if in junctions section
				if current_line == []:															# if end of junctions section
					junctions_found = False														# junctions section ends with \n
					continue
				else:
					name, x, y = current_line
					junctions.append(Junction(name=name,
						x=x,
						y=y,
						pressure=unknown_pressure))															
	return [junctions, pipes]

def create_graph() -> nx.classes.graph.Graph:
	"""
	Creates a NetworkX graph object from lists of nodes and edges taken from .inp file
	Arguments: 	nodes - list of Junction() objects
				edges - list of Pipe() objects
	Returns:	NetworkX graph (networkx.classes.graph.Graph)
	"""

	print("Creating a NetworkX graph from .inp file (networkx_graph.py - create_graph())")

	# get data from .inp file
	nodes, edges = get_node_data(data_file)

	# remove junctions with no edges
	nodes = [i for i in nodes if i.name not in junctions_with_no_edges]

	# create a graph object
	G = nx.Graph()

	# add junctions (nodes), specify their coordinates
	for node in nodes:
		G.add_node(node.name, pos=(float(node.x), float(node.y)))

	# add pipes (edges)
	for edge in edges:
		G.add_edge(edge.node1, edge.node2,
			weight=float(edge.length), name=edge.name)

	return G 

def get_sim_results_LEAK(node: str, area: float, start_time: int, end_time: int):
	"""
	Runs and returns a hydraulic simulation WNTR object (WITH A LEAK)
	Arguments:	node - node which will have the leak (str)
				area - area of the leak in (float, meters squared)
				start_time - when the leak starts (int, hours)
				end_time - when the leak ends (int, hours)
	Returns:	Simulation results (wntr.sim.results.SimulationResults)
	"""
	wn_leaks = wntr.network.WaterNetworkModel(data_file)
	leak_node = wn_leaks.get_node(node)
	leak_node.add_leak(wn_leaks, area=area, start_time=start_time*3600, end_time=end_time*3600)
	sim_leaks = wntr.sim.WNTRSimulator(wn_leaks)
	return sim_leaks.run_sim()

In [56]:
G = create_graph()

leak_area = 0.0005
leak_start = 118
leak_end = 270
for node in G.nodes():
    sim_results_L = get_sim_results_LEAK(node=node, area=leak_area, start_time=leak_start, end_time=leak_end)
    final = pd.concat([sim_results_L.node['pressure'], sim_results_L.link['flowrate']], axis=1)
    final.to_csv(os.path.join(os.getcwd(),'simulations', 'leaks', '000.5start118end270', f'{node}.csv'), index=False)

Creating a NetworkX graph from .inp file (networkx_graph.py - create_graph())


KeyError: 'T1'